In [1]:
region_name = "ALL_10-800_LOOSE"#"TDS" # _ELAISN1
probabilities = [0.002699796063,0.000063342484, 0.000000573303]  # 3 sigma 0.002699796063, 4 sigma 0.000063342484, 5 sigma 0.000000573303
#srcs_ids = [8215 ,32448 ,54184 ,98746 ,151796 ,166179 ,172775 ,193067 ,199832 ,259271 ,265150 ,297278 ,305192 ,388172 ,432606 ,472623 ,535864 ,541266 ,625693 ,1521738 ,1551422 ,2136829 ,] # WD ALL_10-800-LOOSE
srcs_ids = [4993 ,13580 ,14504 ,30475 ,50388 ,52994 ,79477 ,120265 ,138783 ,139480 ,162697 ,204419 ,221349 ,228958 ,236839 ,253214 ,
            253670 ,256433 ,261213 ,331466 ,340464 ,381336 ,390864 ,489430 ,498065 ,525310 ,532554 ,533185 ,540235 ,583838 ,639720 ,
            645215 ,645730 ,659787 ,666224 ,673175 ,680119 ,691408 ,714140 ,764827 ,1037284 ,1069077 ,1070938 ,1805505 ,1839206
            ,2527679 ,3074917 ,] # WD ALL_10-800-LOOSE
srcs_ids = [227 ,243 ,285 ,309]

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import time

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
from astropy.timeseries import LombScargle,TimeSeries,BinnedTimeSeries
from astropy.units import cds
from astropy.table import QTable
from astropy.time import Time
from statsmodels.tsa.tsatools import detrend
from obspy.signal.detrend import spline

import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from loguru import logger
logger.enable("vasca")

#cds.enable()  
import numpy as np

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux, sel_sources, dd_filter2id

region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"
rg = Region()
rg.load_from_fits(region_fname)
#rg.remove_table("tt_coadd_detections")
#rg.tt_coadd_sources.remove_columns(["nr_det","obs_filter_id","flux_nxv","flux_var","flux_cpval","flux_rchiq","fd_src_id","assoc_id","assoc_dist","assoc_ffactor","assoc_fdiff_s2n","rg_fd_id","nr_fd_dets","pos_xv","pos_var","pos_cpval","pos_rchiq"])
#rg.tt_sources.remove_columns(["fd_src_id","rg_fd_id"])
#rg.write_to_fits(region_fname)
rg.tt_sources.add_index("rg_src_id")
src_idx = rg.tt_sources.loc_indices[srcs_ids]
tt_src = rg.tt_sources[src_idx]
display(tt_src)

fd_src_id,nr_det,ra,dec,pos_err,pos_xv,pos_var,pos_cpval,pos_rchiq,coadd_src_id,coadd_dist,obs_filter_id,sel,flux,flux_err,flux_nxv,flux_var,flux_cpval,flux_rchiq,coadd_ffactor,coadd_fdiff_s2n,rg_fd_id,rg_src_id,nr_fd_srcs,hr,hr_err,simbad_match_id,gaiadr3_match_id
,,deg,deg,arcsec,arcsec2,arcsec2,,,,arcsec,,,1e-06 Jy,1e-06 Jy,,1e-12 Jy2,,,,,,,,,,,
int32,int32[2],float64,float64,float32,float32,float32,float32,float32,int64,float32,int32[2],bool,float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32,float32,int64,int32,int32,float32,float32,int32,int32
-1,248 .. 144,150.19491669966794,2.0679206386322235,0.034894396,-0.33851302,0.18164578,1.0,0.3224277,959299,0.16131246,1 .. 2,True,14.191161 .. 8.757307,0.06079538 .. 0.09396652,0.0106525235 .. 0.005538589,3.7525566 .. 2.5326197,0.0 .. 1.772165e-13,3.841772 .. 2.1152527,-100.0,-10000.0,-1,227,10,0.6072997,0.0076302565,0,0
-1,252 .. 147,150.17992517033275,2.110325555655416,0.035408843,-0.24451591,0.30744642,0.99999994,0.566374,75245,0.13007154,1 .. 2,True,17.587757 .. 6.551935,0.06531105 .. 0.0799764,0.002130574 .. -0.0008621514,2.4555638 .. 1.3940743,1.3690256e-15 .. 0.583465,1.8763659 .. 0.9710373,-100.0,-10000.0,-1,243,10,0.3645722,0.0049660443,1,1
-1,244 .. 114,150.14575215591634,2.0430760561265466,0.03967842,-0.38290083,0.27473727,1.0,0.43058085,75401,0.11261488,1 .. 2,True,9.884236 .. 3.657634,0.05455299 .. 0.07133999,0.031847723 .. 0.033267822,4.287245 .. 1.2234199,0.0 .. 2.4905885e-06,6.90565 .. 1.7280945,-100.0,-10000.0,-1,285,10,0.36859575,0.008322803,3,2
-1,216 .. 31,150.12385136844162,2.358249206025685,0.05189303,-0.37029192,0.38448596,1.0,0.518402,75516,0.39484766,1 .. 2,True,4.4475236 .. 1.7711709,0.04511629 .. 0.09256162,0.02649166 .. 0.00075723574,1.0877458 .. 0.29965115,1.6119977e-28 .. 0.3227426,2.4625556 .. 1.099846,-100.0,-10000.0,-1,309,10,0.35776442,0.020704556,4,-1


In [3]:
def freq2period(ff):
    return 1/ff

def period2freq(pp):
    return 1/pp

dd_lcs = rg.get_light_curve(rg_src_ids=srcs_ids)

def plot_LombScargle(rg_src_id):

    #Get source and light curve
    tc_src = rg.get_src_from_id(rg_src_id, load_from_file = False, write_to_file=False, add_sed=False, add_gphoton=False, add_spectrum=False)
    tt_lc = tc_src.tt_source_lc

    fig_lc, axs_lc = plt.subplots(1, 1, figsize=(14, 6), num="LC", clear=True)
    vvis.plot_light_curve(tc_src, ax = axs_lc)
    plt.show()
    
    #Prepare figure
    fig, axs = plt.subplots(1, 2, figsize=(14, 6), num="LombScargle", clear=True)
    axs = axs.flatten()
    panel = 0
    
    for flt in ["NUV","FUV"]:
    
        sel = np.array((tt_lc["obs_filter"]==flt) * (tt_lc["sel"]==True), dtype=bool)
        if sel.sum()<11:
            panel +=1
            continue
    
        #Get LS parameters
        dt = tt_lc["time"][sel][1:]-tt_lc["time"][sel][0:-1]
        t_min = np.min(tt_lc["time"][sel].quantity)
        t_max = np.max(tt_lc["time"][sel].quantity)
        dt_tot =  t_max - t_min
        dt_min = np.min(dt.quantity)
        f_min = 4/(dt_tot)
        f_max = 1/(4*dt_min)
#        f_min = (f_min + f_max)/4
    
        # Run LS
        ls = LombScargle(tt_lc["time"][sel], tt_lc["flux"][sel],tt_lc["flux_err"][sel]) #normalization{‘standard’, ‘model’, ‘log’, ‘psd’},
        freq,power = ls.autopower(minimum_frequency=f_min,maximum_frequency=f_max) #nyquist_factor=5
        
        lg_freq = np.log10(freq)
        lg_power = np.log10(power)
        np.polyfit(lg_freq, lg_power)

        
        #df_ls = pd.DataFrame(data={"power":np.log10(power),"freq":freq})
        #display(df_ls)
        #rolling_power = df_ls["power"].rolling(window=100).mean()
        #power_detrend = (df_ls["power"] - rolling_power)/rolling_power
        #power_detrend = 0.5*power_detrend/np.max(power_detrend)      
        #power_detrend = np.power(10,detrend(df_ls["power"], order=1))
        #print(power_detrend)
        #print(freq)
        #print(freq[1:]-freq[:-1])
        #Get confidence interval
        conf = ls.false_alarm_level(probabilities)
    
        #Get some basic results
        p_peak = power.max()
        f_peak = freq[np.argmax(power)]
        Pval =ls.false_alarm_probability(p_peak)

                #Print basic info
        print(flt,":")
        print(f" Lightcurve total duration= {dt_tot} ({1/dt_tot}), minimum bin distance {dt_min} ({1/dt_min})")
        print(f" Freq_min= {freq[0]}, freqmax= {freq[-1]}, number of freq bins {len(freq)}")
        print(f" Maximum power at frequency {f_peak }, period {1/f_peak}, whith a false probability of {Pval}")

        if flt == "NUV":
            f_peak_nuv = f_peak
        else:
            p_fuv_fnuv = ls.power(f_peak_nuv)
            Pval_fuv_fnuv = ls.false_alarm_probability(p_fuv_fnuv)
            print(f" Power at {f_peak_nuv} is {p_fuv_fnuv}, whith a false probability of {Pval_fuv_fnuv}")
        
        #Plot power spectrum
        axs[panel].axhline(conf[0],linewidth=0.5, ls="--", color='k', label="3/4/5 sigma")
        axs[panel].axhline(conf[1],linewidth=0.5, ls="--", color='k')
        axs[panel].axhline(conf[2],linewidth=0.5, ls="--", color='k')
        #axs[panel].plot(freq,power_detrend, alpha=0.5)
        axs[panel].plot(freq, power, alpha=0.5)

    
        #axs[panel].set_xscale("log")
        #axs[panel].set_yscale("log")
        axs[panel].set_xlabel('Frequency [1/day]')
        axs[panel].set_ylabel('LS power')
        secax = axs[panel].secondary_xaxis("top", functions=(freq2period, period2freq))
        secax.set_xlabel("Period [day]")
        axs[panel].legend()

        #Plot model on lc
        if Pval < probabilities[0]:
            t_fit = np.linspace(t_min,t_max,1000)
            y_fit = ls.model(t_fit, f_peak)
            axs_lc.plot(t_fit, y_fit, alpha=0.5)

        panel +=1
    plt.show()


    
    # fig = plt.figure(figsize=(14, 6))
    #fig_lc, axs_lc = plt.subplots(1, 1, figsize=(14, 6), num="LC", clear=True)
    #axs_lc = axs_lc.flatten()
    #vvis.plot_light_curve(tc_src, ax = axs_lc)
    #vvis.plot_light_curve(rg,rg_src_ids = rg_src_id, ax = axs_lc)

        #t_fit = lc["time"] #np.linspace(0, 1)*uu.d

    #axs[1].set_xlabel('Time (days)')
    #axs[1].set_ylabel('Flux')


    #plt.show()


widgets.interactive(plot_LombScargle, rg_src_id=srcs_ids)

interactive(children=(Dropdown(description='rg_src_id', options=(227, 243, 285, 309), value=227), Output()), _…

In [4]:
# lcs: 0.3452482223510742 # 42 srcs
# Loading: 0.08002376556396484
# LS calculation: 0.012468338012695312
# False prob. calc: 0.0005886554718017578
# Total: 0.43832898139953613

#     best_frequency = frequency[np.argmax(power)]





    # Create time series for folding
    #lc["time"]=Time(lc["time"].quantity, format="mjd")
    # #ts = TimeSeries(data=lc)
    
    #Show best fit model
    # plt.sca(axs[1])
    # ts_folded = ts.fold(period=1/f_peak, epoch_time=lc["time"][0])
    # plt.plot(ts_folded.time.jd, ts_folded['flux'], 'k.', markersize=4)
    # plt.xlabel('Time (days)')
    # plt.ylabel('Flux')
    
    # time_fit = np.linspace(0, 1)*uu.d
    # flux_fit = ls.model(time_fit*uu.d, f_peak)
    # plt.sca(axs[1])
    # plt.plot(time_fit,flux_fit,"ob")
    # plt.show()